In [1]:
%pip install --quiet pandas numpy matplotlib scikit-learn keras tensorflow

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\dcpor\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
# Load the dataset
print("Librerias cargadas correctamente")

Librerias cargadas correctamente


## Carga y exploración del dataset

#### Qué hace:
Carga el dataset de intenciones de compra en línea y muestra los primeros registros y distribución de la variable objetivo (Revenue).

#### Técnicas:
Lectura de CSV con pandas, análisis exploratorio simple.

In [13]:
# Cargar dataset
df = pd.read_csv("online_shoppers_intention.csv")

# Ver primeros registros
print(df.head())

# Revisar clases objetivo
print(df['Revenue'].value_counts())

   Administrative  Administrative_Duration  Informational  \
0               0                      0.0              0   
1               0                      0.0              0   
2               0                      0.0              0   
3               0                      0.0              0   
4               0                      0.0              0   

   Informational_Duration  ProductRelated  ProductRelated_Duration  \
0                     0.0               1                 0.000000   
1                     0.0               2                64.000000   
2                     0.0               1                 0.000000   
3                     0.0               2                 2.666667   
4                     0.0              10               627.500000   

   BounceRates  ExitRates  PageValues  SpecialDay Month  OperatingSystems  \
0         0.20       0.20         0.0         0.0   Feb                 1   
1         0.00       0.10         0.0         0.0   Feb   

## Preprocesamiento de datos

#### Qué hace:
Convierte etiquetas a enteros, codifica variables categóricas, divide en entrenamiento/test y normaliza los datos.

#### Técnicas:

- Codificación One-Hot (get_dummies)

- Escalado Z (StandardScaler)

- Separación de conjuntos (train_test_split)

In [14]:
# Convertir etiquetas a 0/1
df['Revenue'] = df['Revenue'].astype(int)

# Codificar variables categóricas
df = pd.get_dummies(df)

# Separar variables y target
X = df.drop('Revenue', axis=1)
y = df['Revenue']

# Dividir en training/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalado
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Red Neuronal Multicapa (MLP)

## Qué hace:
Define y entrena una red neuronal para clasificación binaria.

## Técnicas:

- Red densa (MLP)

- Función de activación ReLU y sigmoid

- Optimización con Adam y pérdida binaria

- Validación durante el entrenamiento

In [17]:
# Modelo NN
model = Sequential([
    Dense(64, input_shape=(X_train.shape[1],), activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)


Epoch 1/10


c:\Users\dcpor\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


278/278 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8559 - loss: 0.3815 - val_accuracy: 0.8804 - val_loss: 0.2806
Epoch 2/10
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8908 - loss: 0.2666 - val_accuracy: 0.8825 - val_loss: 0.2651
Epoch 3/10
278/278 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9056 - loss: 0.2362 - val_accuracy: 0.8855 - val_loss: 0.2572
Epoch 4/10
278/278 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9058 - loss: 0.2277 - val_accuracy: 0.8875 - val_loss: 0.2555
Epoch 5/10
278/278 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9064 - loss: 0.2292 - val_accuracy: 0.8926 - val_loss: 0.2491
Epoch 6/10
278/278 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9080 - loss: 0.2196 - val_accuracy: 0.8936 - val_loss: 0.2542
Epoch 7/10
278/278 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9089 - loss: 0.2193 - val_accuracy: 0.8926 - val_loss: 0.2490
Epoch 8/10
278/278 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9129 - loss: 0.2117 - val_accuracy: 0.8875 - val_

## Evaluación del modelo NN

#### Qué hace:
Evalúa el modelo sobre el conjunto de test.

#### Técnicas:
Evaluación final con métricas estándar (accuracy y loss)

In [18]:
# Evaluar
loss, acc = model.evaluate(X_test, y_test)
print(f"Accuracy NN: {acc:.4f}")

78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8901 - loss: 0.2575
Accuracy NN: 0.8913


## Predicción de nuevo sample

#### Qué hace:
Define una función para predecir nuevos ejemplos individuales con cualquier modelo (RF o NN), asegurando que el input sea transformado igual que los datos de entrenamiento.

#### Técnicas:

- Manipulación de diccionarios/DataFrames

- Transformación consistente

- Generalización de predicción

In [19]:
def predecir_sample(sample_dict, modelo, scaler):
    sample_df = pd.DataFrame([sample_dict])
    sample_df = pd.get_dummies(sample_df)
    sample_df = sample_df.reindex(columns=X.columns, fill_value=0)
    scaled = scaler.transform(sample_df)
    pred = modelo.predict(scaled)
    return pred[0]

# Ejemplo con NN
nn_pred = model.predict(scaler.transform([list(nuevo_sample.values())]))
print("Predicción NN:", nn_pred[0][0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Predicción NN: 0.0003476741


c:\Users\dcpor\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
